In [ ]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import platform
plt.style.use('seaborn')
from datetime import datetime
import json

from metric import E1_loss, E2_loss, total_loss
from models import classifier, cnn_model, conv_block, cnn_parallel
from utils import train_model, eval_model, dfDataset, weights_init

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

from sklearn.model_selection import KFold, train_test_split
from torchsummary import summary

### Configuration

In [ ]:
EPOCH = 100
base_lr = 0.001
now = datetime.strftime(datetime.now(), '%Y%m%d-%H%M%S')
save_path = './model/{}'.format(now)
initialize = True
print_summary = True
batch_size = 256
nfold = 10

### load dataset

In [ ]:
if platform.system() == 'Windows':
    root_dir = 'D:/datasets/KAERI_dataset/'
else:
    root_dir = '/home/bskim/project/kaeri/KAERI_dataset/'

train_f = pd.read_csv(os.path.join(root_dir, 'train_features.csv'))
train_t = pd.read_csv(os.path.join(root_dir, 'train_target.csv'))
test_f = pd.read_csv(os.path.join(root_dir, 'test_features.csv'))

train_f = train_f[['Time','S1','S2','S3','S4']].values
train_f = train_f.reshape((-1, 1, 375, 5))#.astype(np.float32)

test_f = test_f[['Time','S1','S2','S3','S4']].values
test_f = test_f.reshape((-1, 1, 375, 5))#.astype(np.float32)
test_f = torch.FloatTensor(test_f)

### Train

In [ ]:
loss_per_model = {}
for name in ['XY','M','V']:
    print('{} train...'.format(name))
    
    # make dataset
    train_target = train_t[list(name)].values#.astype(np.float32)

    # trainx, valx, trainy, valy = train_test_split(train_f, train_target, test_size = 0.2, shuffle = True, random_state = 38)
    fold = KFold(nfold, shuffle = True, random_state= 25)
    loss_per_cv = []
    for i, (train_idx, val_idx) in enumerate(fold.split(train_f, y = train_target)):
        print('fold {}'.format(i+1))
        trainx = train_f[train_idx]
        valx = train_f[val_idx]
        trainy = train_target[train_idx]
        valy = train_target[val_idx]
        
        train_dataset = dfDataset(trainx.astype(np.float32), trainy)
        train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
        val_dataset = dfDataset(valx.astype(np.float32), valy)
        val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

        #fc = classifier([128, 64, 32, 16], input_size = 256*2*5, output_size = len(name))
        #conv = conv_block([32, 64, 128, 256, 256, 256, 256], [1, 375, 5], (3, 1))
        conv = conv_block([16, 32, 64, 128, 256, 256, 256], [1, 375, 5], (3, 1))
        fc = classifier([128, 64, 32, 16], input_size = 256*1*5, output_size = len(name))
        # define model
        model = cnn_model(conv, fc)
        optimizer = torch.optim.Adam(model.parameters(), lr = base_lr)

        if name == 'XY':
            criterion = E1_loss
        else:
            criterion = E2_loss

        model = model.cuda()
        if initialize:
            model.apply(weights_init)

        curr_loss = 1e+7
        os.makedirs(save_path) if not os.path.exists(save_path) else None
        #train
        for ep in range(1, EPOCH + 1):
            model.train()
            loss = train_model(model, train_loader, criterion, optimizer, criterion)
            model.eval()
            val_loss =eval_model(model, val_loader, criterion)
            if curr_loss > val_loss:
                print('[{}] : train loss {:4f}, val loss drop {:.4f} to {:.4f}'.format(ep, np.mean(loss), curr_loss, val_loss))
                curr_loss = val_loss
                torch.save(model.state_dict(), os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i+1)))
        loss_per_cv.append(curr_loss)
    loss_per_model[name] = loss_per_cv           
            #else:
                #print('[{}] : train loss {:.4f}, val loss {:.4f}, not drop'.format(ep, np.mean(loss), val_loss))

In [ ]:
with open(os.path.join(save_path, 'loss_info.json'), 'w') as f:
    for k in loss_per_model:
        loss_per_model[k] = np.mean(loss_per_model[k])
    f.write(json.dumps(loss_per_model))

## test

In [ ]:
submission = pd.read_csv(os.path.join(root_dir, 'sample_submission.csv'))
for name in ['XY','M','V']:
    fc = classifier([128, 64, 32, 16], input_size = 256*1*5, output_size = len(name))
    conv = conv_block([16, 32, 64, 128, 256, 256, 256], [1, 375, 5], (3, 1))
    # define model
    model = cnn_model(conv, fc)
    pred_array = []
    for i in range(1, nfold + 1):
        model.load_state_dict(torch.load(os.path.join(save_path, 'model_{}_fold{}.pt'.format(name, i))))
        model = model.cuda()

        with torch.no_grad():
            predict = model(test_f.cuda())
        pred_array.append(predict.detach().cpu().numpy())
    submission[list(name)] = np.mean(pred_array, axis = 0)

In [ ]:
submission.head()

In [ ]:
submission.to_csv(os.path.join(save_path, 'submit.csv'), index = False)